In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import delfi.distribution as dd
import numpy as np
import pickle
import time
import scipy.stats as st
import os 
from lfimodels.balancednetwork.BalancedNetworkSimulator import BalancedNetwork
from lfimodels.balancednetwork.BalancedNetworkStats import BalancedNetworkStats

In [ ]:
path_to_save_folder = 'figures/'
save_figure = True

inference_method = 'basic'
fileformat = '.png'
dpi = 300
filenames = ['1509381975706594_basic_Jee_ntrain400']
param_name = r'$J_{ee}$'

In [ ]:
plt.figure(figsize=(15, 5))
legend_size = 13
label_size = 16

for idx, filename in enumerate(filenames): 
    time_str = filename[:filename.find('_')]
    fullname = 'data/' + filename + '.p'

    # load data 
    with open(fullname, 'rb') as handle:
        result_dict = pickle.load(handle)

    # unpack values 
    true_params, stats_obs, nrounds, ntrain, posterior, out, trn_data, prior = result_dict.values()
    assert len(true_params) == 1, 'this notebook is for inference on 1 parameter: len(params)'

    nrounds = len(out)

    # extract the posterior 
    n_components = len(posterior.a)
    means = [posterior.xs[c].m for c in range(n_components)]
    Ss = [posterior.xs[c].S for c in range(n_components)]

    theta = np.linspace(prior.lower[0], prior.upper[0], 1000)
    sub_means = [[means[c][0]] for c in range(n_components)]
    sub_cov = np.asarray([Ss[c] for c in range(n_components)])
    pdf = dd.mixture.MoG(a=posterior.a, ms=sub_means, Ss=sub_cov)
    post_pdf = pdf.eval(theta[:, np.newaxis], log=False)

    plt.subplot(1, len(filenames), idx + 1)
    plt.plot(theta, post_pdf, label='posterior')
    plt.axvline(x=true_params[0], label=r'observed {}'.format(param_name), linestyle='--', color='C1')
    plt.legend(prop=dict(size=legend_size))
    plt.xlabel(param_name, fontsize=label_size)
    if idx == 0: 
        plt.ylabel(r'$\hat{p}( \theta | x=x_{o})$', fontsize=label_size)
    
if len(filenames) > 1: 
    filename = time_str + '_combined_results'
else: 
    filename = time_str + '_{}_posterior_r{}_ntrain{}'.format(inference_method, nrounds, ntrain)

plt.tight_layout()

if save_figure and os.path.exists(path_to_save_folder): 
    plt.savefig(os.path.join(path_to_save_folder, filename + fileformat), dpi=dpi);

## Posterior predictive checking 

Generate samples from the posterior and simulate them. The resulting data should be near the observed data. 

In [ ]:
m = BalancedNetwork(inference_param='wee', n_servers=1, duration=3., first_port=8010, 
                    save_raster_plots=True, 
                    save_folder='/Users/Jan/Dropbox/Master/mackelab/code/lfi-experiments/balancednetwork/figures/simulation_raster_plots/')
s = BalancedNetworkStats(n_workers=2)

In [ ]:
# calculate mean of mixture 
mean = np.sum([a * m[0] for a, m in zip(posterior.a, means)])
# calculate variance of mixture 
sum_squared = np.sum([a * m[0]**2 for a, m in zip(posterior.a, means)])
squared_sum = mean**2
std = np.sum([a * Ss[0][0] for a, m in zip(posterior.a, Ss)]) + sum_squared - squared_sum

In [ ]:
m.start_server()
# generate theta +-3, 2, 1 0 stds away from mode
thetas = [mean + i * std for i in [0]]
sum_stats = []
# simulate and collect sum stats
data = m.gen(thetas)

In [ ]:
sum_stats = s.calc_all(data)

In [ ]:
# plot the resulting stats with the observed stats 
plt.figure(figsize=(8, 4))
sum_stats = np.array(sum_stats).squeeze()
plt.axvline(x=mean, linestyle='--', color='C4')
plt.axvline(x=true_params, linestyle='--', color='C5')

plt.plot(thetas, sum_stats, 'o')
#plt.plot(true_params, stats_obs, '*')

plt.legend(['posterior mean', 'true theta', 'ff1', 'rate mean', 'kurtosis', 'rel corr'])
plt.xlabel('theta')
plt.ylabel('stats')
plt.title('Summary stats +-3 std around the posterior mean')

filename = '{}_ppch_r{}_ntrain{}'.format(inference_method, nrounds, ntrain)
plt.tight_layout()
if save_figure and os.path.exists(path_to_save_folder): 
    plt.savefig('figures/' + filename + fileformat, dpi=dpi);

In [ ]:
thetas

In [ ]:
sum_stats[0].shape

## Compare only simulation of mean posterior vs. original parameters

In [ ]:
jee = posterior.mean[0]
stats = sum_stats[0].flatten()

In [ ]:
labels = ['FF', 'rate', 'kurtosis', '+ correlation', 'FF inh', 'rate inh', 'kurtosis inh', '+ correlation inh']
plt.figure(figsize=(15, 5))
plt.plot(stats, '-o', label='posterior mean')
ax = plt.plot(stats_obs.T, '-o', label='observed')
plt.xticks(np.arange(8), labels)
plt.gca().tick_params(axis='x', labelsize=15)
plt.legend(prop=dict(size=legend_size + 2))

filename = '{}_ppch_ntrain{}_{}'.format(inference_method, ntrain, param_name)

plt.tight_layout()
if save_figure and os.path.exists(path_to_save_folder): 
    plt.savefig('figures/' + filename + fileformat, dpi=dpi);

In [ ]:
plt.savefig?